In [12]:
import findspark
findspark.init()

In [13]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [14]:
#obj3 = spark.read.csv("C:\data\FINAL-2\objetivo3-final.csv",header=True, inferSchema=True, nullValue="NA")
obj3=spark.read.format("csv") \
    .option("quote", "") \
    .option("escape", "\"") \
    .option("header", "true") \
    .option("multiLine", "true") \
    .option("encoding","windows-1252") \
    .load("C:\data\FINAL-3\objetivo3-final.csv")

# Selección de datos

In [15]:
#Objetivo 1 - Eliminando columnas
obj3 = obj3.drop('id','ApellidoPaterno','ApellidoMaterno','Nombres','CategoriaEnfermedad', 'Peso', 'Talla','Departamento','Zona','Mes','Anio')
obj3.show(10)

+----+-------------------+------------------+-----------------+----------------------+
|Edad|AntecedentesFamilia|RangoGrasaVisceral|AdiccionDepresora|SubcategoriaEnfermedad|
+----+-------------------+------------------+-----------------+----------------------+
|  25|                 NO|         SALUDABLE|DROGA ALUCINOGENA|  I26 - EMBOLIA PUL...|
|  52|                 NO|         PELIGROSO|          NINGUNO|  I21 - INFARTO AGU...|
|  49|                 NO|         PELIGROSO|          ALCOHOL|  I06 - ENFERMEDADE...|
|  61|                 NO|         SALUDABLE|          NINGUNO|  I34 - TRASTORNOS ...|
|  38|                 NO|         PELIGROSO|          ALCOHOL|  I06 - ENFERMEDADE...|
|  22|                 NO|         SALUDABLE|       TABAQUISMO|   VALVULA NO ESPEC...|
|  17|                 NO|         PELIGROSO|          NINGUNO|  I66 - OCLUSION Y ...|
|  89|                 SI|         SALUDABLE|          ALCOHOL|   I46 - PARO CARDIACO|
|  76|                 NO|         SALUDABL

# Limpieza de datos

In [16]:
#Eliminando valores que no corresponden del dataset
obj3 = obj3.dropna()
obj3 = obj3.filter('Edad > 0')
obj3 = obj3.where((obj3.AdiccionDepresora == 'NINGUNA') | (obj3.AdiccionDepresora == 'TABAQUISMO') | (obj3.AdiccionDepresora == 'DROGA ALUCINOGENA'))
obj3 = obj3.where((obj3.RangoGrasaVisceral == 'SALUDABLE') | (obj3.RangoGrasaVisceral == 'PELIGROSO'))
obj3.show(10)

+----+-------------------+------------------+-----------------+----------------------+
|Edad|AntecedentesFamilia|RangoGrasaVisceral|AdiccionDepresora|SubcategoriaEnfermedad|
+----+-------------------+------------------+-----------------+----------------------+
|  25|                 NO|         SALUDABLE|DROGA ALUCINOGENA|  I26 - EMBOLIA PUL...|
|  22|                 NO|         SALUDABLE|       TABAQUISMO|   VALVULA NO ESPEC...|
|  76|                 NO|         SALUDABLE|DROGA ALUCINOGENA|  I32 - PERICARDITI...|
|  55|                 NO|         SALUDABLE|DROGA ALUCINOGENA|  I64 - ACCIDENTE V...|
|  17|                 SI|         PELIGROSO|DROGA ALUCINOGENA|  I87 - OTROS TRAST...|
|  84|                 SI|         PELIGROSO|       TABAQUISMO|  I36 - TRASTORNOS ...|
|  61|                 SI|         PELIGROSO|       TABAQUISMO|  I34 - TRASTORNOS ...|
|  75|                 NO|         SALUDABLE|       TABAQUISMO|  I85 - VARICES ESO...|
|  45|                 NO|         SALUDABL

57676

# Integración de datos

In [17]:
from pyspark.ml.feature import StringIndexer

obj3 = StringIndexer(inputCol='AntecedentesFamilia', outputCol= 'ant_idx').fit(obj3).transform(obj3)
obj3 = StringIndexer(inputCol='RangoGrasaVisceral', outputCol= 'rgb_idx').fit(obj3).transform(obj3)
obj3 = StringIndexer(inputCol='AdiccionDepresora', outputCol= 'adc_idx').fit(obj3).transform(obj3)
obj3 = StringIndexer(inputCol='Edad', outputCol= 'edad_idx').fit(obj3).transform(obj3)


In [18]:
obj3.show(10)

+----+-------------------+------------------+-----------------+----------------------+-------+-------+-------+--------+
|Edad|AntecedentesFamilia|RangoGrasaVisceral|AdiccionDepresora|SubcategoriaEnfermedad|ant_idx|rgb_idx|adc_idx|edad_idx|
+----+-------------------+------------------+-----------------+----------------------+-------+-------+-------+--------+
|  25|                 NO|         SALUDABLE|DROGA ALUCINOGENA|  I26 - EMBOLIA PUL...|    1.0|    1.0|    0.0|    55.0|
|  22|                 NO|         SALUDABLE|       TABAQUISMO|   VALVULA NO ESPEC...|    1.0|    1.0|    1.0|    82.0|
|  76|                 NO|         SALUDABLE|DROGA ALUCINOGENA|  I32 - PERICARDITI...|    1.0|    1.0|    0.0|    53.0|
|  55|                 NO|         SALUDABLE|DROGA ALUCINOGENA|  I64 - ACCIDENTE V...|    1.0|    1.0|    0.0|    46.0|
|  17|                 SI|         PELIGROSO|DROGA ALUCINOGENA|  I87 - OTROS TRAST...|    0.0|    0.0|    0.0|    26.0|
|  84|                 SI|         PELIG

In [19]:
obj3 = StringIndexer(inputCol='SubcategoriaEnfermedad', outputCol= 'label').fit(obj3).transform(obj3)
obj3.show(10)

+----+-------------------+------------------+-----------------+----------------------+-------+-------+-------+--------+-----+
|Edad|AntecedentesFamilia|RangoGrasaVisceral|AdiccionDepresora|SubcategoriaEnfermedad|ant_idx|rgb_idx|adc_idx|edad_idx|label|
+----+-------------------+------------------+-----------------+----------------------+-------+-------+-------+--------+-----+
|  25|                 NO|         SALUDABLE|DROGA ALUCINOGENA|  I26 - EMBOLIA PUL...|    1.0|    1.0|    0.0|    55.0| 27.0|
|  22|                 NO|         SALUDABLE|       TABAQUISMO|   VALVULA NO ESPEC...|    1.0|    1.0|    1.0|    82.0| 55.0|
|  76|                 NO|         SALUDABLE|DROGA ALUCINOGENA|  I32 - PERICARDITI...|    1.0|    1.0|    0.0|    53.0| 50.0|
|  55|                 NO|         SALUDABLE|DROGA ALUCINOGENA|  I64 - ACCIDENTE V...|    1.0|    1.0|    0.0|    46.0| 14.0|
|  17|                 SI|         PELIGROSO|DROGA ALUCINOGENA|  I87 - OTROS TRAST...|    0.0|    0.0|    0.0|    26.0

In [20]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = ['ant_idx','rgb_idx','adc_idx','edad_idx'],outputCol='features' )
obj3 = assembler.transform(obj3)

In [21]:
obj3.show(10)

+----+-------------------+------------------+-----------------+----------------------+-------+-------+-------+--------+-----+------------------+
|Edad|AntecedentesFamilia|RangoGrasaVisceral|AdiccionDepresora|SubcategoriaEnfermedad|ant_idx|rgb_idx|adc_idx|edad_idx|label|          features|
+----+-------------------+------------------+-----------------+----------------------+-------+-------+-------+--------+-----+------------------+
|  25|                 NO|         SALUDABLE|DROGA ALUCINOGENA|  I26 - EMBOLIA PUL...|    1.0|    1.0|    0.0|    55.0| 27.0|[1.0,1.0,0.0,55.0]|
|  22|                 NO|         SALUDABLE|       TABAQUISMO|   VALVULA NO ESPEC...|    1.0|    1.0|    1.0|    82.0| 55.0|[1.0,1.0,1.0,82.0]|
|  76|                 NO|         SALUDABLE|DROGA ALUCINOGENA|  I32 - PERICARDITI...|    1.0|    1.0|    0.0|    53.0| 50.0|[1.0,1.0,0.0,53.0]|
|  55|                 NO|         SALUDABLE|DROGA ALUCINOGENA|  I64 - ACCIDENTE V...|    1.0|    1.0|    0.0|    46.0| 14.0|[1.0,

In [22]:
obj3_train, obj3_test = obj3.randomSplit([0.8, 0.2], seed=23)
[obj3_train.count(), obj3_test.count()]

[46216, 11460]

# Aplicación del modelo de algoritmo

In [23]:
#Regresión logistica Clasificador
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label')
lr_model = lr.fit(obj3_train)

In [24]:
prediction = lr_model.transform(obj3_test)
prediction.select('label', 'prediction', 'probability').show(5)
prediction.groupBy("label", "prediction").count().show()
prediction.show()

+-----+----------+--------------------+
|label|prediction|         probability|
+-----+----------+--------------------+
| 22.0|      14.0|[0.01378457861968...|
| 43.0|      14.0|[0.01378457861968...|
| 51.0|      14.0|[0.01378457861968...|
| 78.0|      14.0|[0.01378457861968...|
| 71.0|      14.0|[0.01378457861968...|
+-----+----------+--------------------+
only showing top 5 rows

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
| 66.0|       5.0|   32|
| 76.0|      10.0|    6|
| 40.0|       2.0|    6|
| 65.0|      20.0|    6|
| 51.0|      20.0|    7|
| 64.0|       5.0|   29|
| 76.0|      13.0|    9|
| 68.0|      11.0|    2|
| 38.0|       1.0|    4|
| 40.0|      10.0|    7|
| 34.0|       2.0|    9|
| 42.0|      23.0|   10|
| 19.0|       5.0|   29|
| 32.0|       4.0|    5|
| 80.0|       6.0|   10|
| 29.0|      75.0|    4|
|  6.0|      50.0|    3|
| 62.0|       0.0|    3|
| 17.0|      23.0|    4|
| 49.0|      24.0|    1|
+-----+----------+-----+
only showing to

In [28]:
from pyspark.sql.functions import round
#Exportar datos pronosticados
#prediction = prediction.withColumn('probability',round(prediction.probability,2))
df_prediction=prediction.drop('features','rawPrediction','probability')
df_prediction.coalesce(1).write.option('header',True).csv('prediccion3.csv')


root
 |-- Edad: string (nullable = true)
 |-- AntecedentesFamilia: string (nullable = true)
 |-- RangoGrasaVisceral: string (nullable = true)
 |-- AdiccionDepresora: string (nullable = true)
 |-- SubcategoriaEnfermedad: string (nullable = true)
 |-- ant_idx: double (nullable = false)
 |-- rgb_idx: double (nullable = false)
 |-- adc_idx: double (nullable = false)
 |-- edad_idx: double (nullable = false)
 |-- label: double (nullable = false)
 |-- prediction: double (nullable = false)



# Evaluación del modelo de algoritmo

In [29]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
eval_df = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='label')
prediction.select('label','prediction')
AUC = eval_df.evaluate(prediction)
print("AUC score is : ",AUC)

AUC score is :  0.4779494202089491


In [30]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

df_accuracy = MulticlassClassificationEvaluator(labelCol="label", metricName="accuracy").evaluate(prediction)
df_accuracy

0.011518324607329843

In [31]:
df_precision = MulticlassClassificationEvaluator(labelCol="label", metricName="weightedPrecision").evaluate(prediction)
df_precision

0.002582026344330823

In [32]:
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label != prediction').count()
FP = prediction.filter('prediction = 1 AND label != prediction').count()


In [33]:
precision = (TN+ TP)/(TN+TP+FN+FP)
print(precision)


0.006160164271047228
